In [3]:
import pandas as pd

In [8]:
df = pd.read_csv(r"C:\Users\othma\PycharmProjects\UAS NLP\coba.csv", delimiter=';')

In [9]:
df.head()

,no,tweet,sentiment
0,1,fatur jelek,positif
1,2,othman ganteng,positif
2,3,sansan jahat,negatif
3,4,sansan baik,positif


In [ ]:
def cleaning(df):
    import re
    import string

    def clean(text):
        text = str(text)
        text = text.lower()
        text = re.sub("@[A-Za-z0-9_]+", "", text)  # Menghapus @<name> [mention twitter]
        text = re.sub("#\w+", "", text)
        text = re.sub("\[.*?\]", "", text)
        text = re.sub("https?://\S+|www\.\S+", "", text)
        text = re.sub("<.*?>+", "", text)
        text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
        # text = re.sub('\n', '', text)
        text = re.sub("\w*\d\w*", "", text)
        text = re.sub("\d+", "", text)
        text = re.sub("\s+", " ", text).strip()
        # text = re.sub('\n', '', text) jadi:
        text = text.replace("\n", " ")
        text = " ".join(text.split())
        return text

    # Applying the cleaning function
    df['tweet2'] = df['Tweet'].apply(lambda x: clean(x))
    df.head()

    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    import swifter

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    df['cleaned'] = df['tweet2'].swifter.apply(lambda x: stemmer.stem(x))

    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

    factory = StopWordRemoverFactory()

    stopword = factory.get_stop_words()
    stopwords = factory.create_stop_word_remover()

    df['cleaned'] = df['cleaned'].apply(lambda x: stopwords.remove(x))

    from nltk import word_tokenize
    from nltk.corpus import stopwords

    new_stopwords = ['sahabat', 'nomor', 'yg', 'jd', 'klo', 'pa', 'bpj', 'nya', 'tp', 'ga', 'jg', 'https', 'co', 'aja',
                     'ya',
                     'gw', 'kalo', 'tuh', 'tau', 'gk', 'gak', 'kalo', 'amp', 'gitu', 'krn', 'dr', 'sih', 'gue', 'bgt',
                     'aja', 'ya', 'krn', 'pake', 'udah', 'sampe', 'udah', 'emang', 'nggak', 'gk', 'udh', 'kela',
                     'bpjskesehatanri', 'duanya', 'banget', 'tdk', 'bpjsmelayani', 'gotongroyongsemuatertolong',
                     'gotongroyongsemuatertolong_jknhadiruntukrakyat', 'bpjskesehatan_bpjsmelayani'
        , 'dgn', 'nih', 'loh', 'dpt', 'yaa', 'dah', 'kak', 'sm', 'ngga', 'dg', 'deh', 'lho', 'utk', 'kali', 'a', 'b',
                     'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                     'w', 'x', 'y', 'z', 'gua', 'sya', 'iya', 'ni']

    stop_words = stopwords.words('indonesian')
    stop_words.extend(new_stopwords)

    def removeStopword(str):
        stop_words = stopwords.words('indonesian')
        stop_words.extend(new_stopwords)
        # stop_words = set(stopwords.words('indonesian'))
        word_tokens = word_tokenize(str)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        return ' '.join(filtered_sentence)

    df['cleaned'] = df['cleaned'].apply(lambda x: removeStopword(x))

    return df


def scraping(maskapai):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    import snscrape.modules.twitter as sntwitter

    # Creating list to append tweet data to

    attributes_container = []

    # Using TwitterS earchScraper to scrape data and append tweets to list
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(
            f'{maskapai} since:{date.today() + relativedelta(months=-2)} until:{date.today()} lang:id').get_items()):  # big mouth:2022-08-05 until:2022-09-06
        if i > 100:
            break
        attributes_container.append(
            [tweet.user.username, tweet.id, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])

    # Creating a dataframe to load the list

    tweet = pd.DataFrame(attributes_container,
                         columns=["User", "Tweet ID", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"])

    tweet.to_csv(f'{maskapai}.csv', index=False)

    return tweet

def load_model_predict(df):
    sentences = df['cleaned'].tolist()

    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from keras.preprocessing.text import Tokenizer

    # The maximum number of words to be used. (most frequent)
    MAX_NB_WORDS = 1000
    # Max number of words in each complaint.
    MAX_SEQUENCE_LENGTH = 50
    # This is fixed.
    EMBEDDING_DIM = 16

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(df['Tweet'].values)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    X = tokenizer.texts_to_sequences(df['Tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    print('Shape of data tensor:', X.shape)

    from tensorflow.keras.models import model_from_json

    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")

    # evaluate loaded model on test data
    loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    y_pred = loaded_model.predict(X)

    df['Sentiment'] = np.NaN
    for i in range(len(y_pred)):
        df['Sentiment'][i] = np.argmax(y_pred[i])

    return df


df = scraping("garuda indonesia")
df = load_model_predict(cleaning(df))
df

NameError: name 'pd' is not defined

In [30]:
df1 = pd.read_csv(r"C:\Users\othma\PycharmProjects\UAS NLP\labellinggg.csv")
df1.describe()

,Tweet ID,labelling
count,7.087000e+03,7084.000000
mean,2.030214e+22,0.763269
std,5.263113e+22,0.676338
min,1.477221e+18,0.000000
25%,1.565143e+18,0.000000
50%,1.580000e+18,1.000000
75%,1.580000e+18,1.000000
max,1.586860e+23,2.000000


In [24]:
import pandas as pd
def cleaning(df):
    import re
    import string

    def clean(text):
        text = str(text)
        text = text.lower()
        text = re.sub("@[A-Za-z0-9_]+", "", text)  # Menghapus @<name> [mention twitter]
        text = re.sub("#\w+", "", text)
        text = re.sub("\[.*?\]", "", text)
        text = re.sub("https?://\S+|www\.\S+", "", text)
        text = re.sub("<.*?>+", "", text)
        text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
        # text = re.sub('\n', '', text)
        text = re.sub("\w*\d\w*", "", text)
        text = re.sub("\d+", "", text)
        text = re.sub("\s+", " ", text).strip()
        # text = re.sub('\n', '', text) jadi:
        text = text.replace("\n", " ")
        text = " ".join(text.split())
        return text

    # Applying the cleaning function
    df['tweet2'] = df['Tweet'].apply(lambda x: clean(x))
    df.head()

    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    import swifter

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    df['cleaned'] = df['tweet2'].swifter.apply(lambda x: stemmer.stem(x))

    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

    factory = StopWordRemoverFactory()

    stopword = factory.get_stop_words()
    stopwords = factory.create_stop_word_remover()

    df['cleaned'] = df['cleaned'].apply(lambda x: stopwords.remove(x))

    from nltk import word_tokenize
    from nltk.corpus import stopwords

    new_stopwords = ['sahabat', 'nomor', 'yg', 'jd', 'klo', 'pa', 'bpj', 'nya', 'tp', 'ga', 'jg', 'https', 'co', 'aja',
                     'ya',
                     'gw', 'kalo', 'tuh', 'tau', 'gk', 'gak', 'kalo', 'amp', 'gitu', 'krn', 'dr', 'sih', 'gue', 'bgt',
                     'aja', 'ya', 'krn', 'pake', 'udah', 'sampe', 'udah', 'emang', 'nggak', 'gk', 'udh', 'kela',
                     'bpjskesehatanri', 'duanya', 'banget', 'tdk', 'bpjsmelayani', 'gotongroyongsemuatertolong',
                     'gotongroyongsemuatertolong_jknhadiruntukrakyat', 'bpjskesehatan_bpjsmelayani'
        , 'dgn', 'nih', 'loh', 'dpt', 'yaa', 'dah', 'kak', 'sm', 'ngga', 'dg', 'deh', 'lho', 'utk', 'kali', 'a', 'b',
                     'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                     'w', 'x', 'y', 'z', 'gua', 'sya', 'iya', 'ni']

    stop_words = stopwords.words('indonesian')
    stop_words.extend(new_stopwords)

    def removeStopword(str):
        stop_words = stopwords.words('indonesian')
        stop_words.extend(new_stopwords)
        # stop_words = set(stopwords.words('indonesian'))
        word_tokens = word_tokenize(str)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        return ' '.join(filtered_sentence)

    df['cleaned'] = df['cleaned'].apply(lambda x: removeStopword(x))

    return df


def scraping(maskapai):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    import snscrape.modules.twitter as sntwitter

    # Creating list to append tweet data to

    attributes_container = []

    # Using TwitterS earchScraper to scrape data and append tweets to list
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(
            f'{maskapai} since:{date.today() + relativedelta(months=-2)} until:{date.today()} lang:id').get_items()):  # big mouth:2022-08-05 until:2022-09-06
        if i > 100:
            break
        attributes_container.append(
            [tweet.user.username, tweet.id, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])

    # Creating a dataframe to load the list

    tweet = pd.DataFrame(attributes_container,
                         columns=["User", "Tweet ID", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"])

    tweet.to_csv(f'{maskapai}.csv', index=False)

    return tweet

def wordcloud(df, sentimen):
    if sentimen == 'positif':
       df = df[(df['Sentiment'] == 2.0)]
    elif sentimen == 'negatif':
        df = df[(df['Sentiment'] == 0.0)]
    else:
        df = df[(df['Sentiment'] == 1.0)]

    # Import the wordcloud library
    from wordcloud import WordCloud
    # Join the different processed titles together.
    long_string = ','.join(map(str, list(df['cleaned'].values)))
    # Create a WordCloud object
    word_cloud = WordCloud(
            width=3000,
            height=2000,
            random_state=1,
            background_color="salmon",
            colormap="Pastel1",
            collocations=False,
            ).generate(long_string)

    import base64
    from PIL import Image
    from io import BytesIO

    buffer = BytesIO()
    word_cloud.to_image().save(buffer, 'png')
    b64 = base64.b64encode(buffer.getvalue())
    # Visualize the word cloud
    return b64


df = scraping("garuda indonesia")
import numpy as np
df = load_model_predict(cleaning(df))

df

Pandas Apply:   0%|          | 0/101 [00:00<?, ?it/s]

Found 975 unique tokens.
Shape of data tensor: (101, 50)
Loaded model from disk
4/4 [==============================] - 4s 8ms/step


<ipython-input-2-080aa584048a>:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'][i] = np.argmax(y_pred[i])


,User,Tweet ID,Date Created,Number of Likes,Source of Tweet,Tweet,tweet2,cleaned,Sentiment
0,roswitwit,1601306520133799938,2022-12-09 20:03:19+00:00,0,Foursquare,I'm at Garuda Indonesia Domestic Check-In Coun...,im at garuda indonesia domestic checkin counte...,im at garuda indonesia domestic checkin counte...,1.0
1,Saju_Solda,1601290027274670081,2022-12-09 18:57:47+00:00,0,Twitter for iPhone,@SiaranBolaLive Maskapai garuda indonesia min.,maskapai garuda indonesia min,maskapai garuda indonesia min,0.0
2,VehiclesDiecast,1601239126648696832,2022-12-09 15:35:32+00:00,0,dlvr.it,1:400 NG Model Garuda Indonesia BOEING 777-300...,ng model garuda indonesia boeing passenger pla...,ng model garuda indonesia boeing passenger pla...,1.0
3,ChakrawalaS,1601238828110446593,2022-12-09 15:34:20+00:00,0,Jetpack.com,KPK Mencecar Politikus Partai Demokrat Atte Su...,kpk mencecar politikus partai demokrat atte su...,kpk cecar politikus partai demokrat atte sugan...,0.0
4,aboutmalangcom,1601233972759265283,2022-12-09 15:15:03+00:00,0,dlvr.it,Apa Itu Pangkat Tituler? Ini 3 Pilot Garuda In...,apa itu pangkat tituler ini pilot garuda indon...,pangkat tituler pilot garuda indonesia pangkat...,0.0
...,...,...,...,...,...,...,...,...,...
96,garuda_cargo,1600407110118182912,2022-12-07 08:29:23+00:00,0,Sociomedio Pro Enterprise,"@pradinayashinta Selamat sore Kak, ada yang in...",selamat sore kak ada yang ingin anda tanyakan ...,selamat sore putar produk cargo garuda indones...,0.0
97,kompascom,1600398203051835393,2022-12-07 07:54:00+00:00,3,Twitter Web App,Garuda Indonesia membuka kembali penerbangan i...,garuda indonesia membuka kembali penerbangan i...,garuda indonesia buka terbang internasional tu...,0.0
98,ekakusumajaya,1600392617023266816,2022-12-07 07:31:48+00:00,0,Twitter for Android,@priyayi_jogja @ilham_mochnoer @idextratime Ga...,garuda indonesia yang mana sih 🤣,garuda indonesia,0.0
99,IndonesiaGaruda,1600375757007917057,2022-12-07 06:24:48+00:00,0,Sociomedio Pro Enterprise,@IntanAyuArimbi Kami nantikan penerbangan Anda...,kami nantikan penerbangan anda bersama garuda ...,terbang garuda indonesia mutia,1.0


In [25]:
a = wordcloud(df, 'positif')

In [26]:
a

b'...'

In [14]:
x = df["Tweet"]
x



0      I'm at Garuda Indonesia Domestic Check-In Coun...
1         @SiaranBolaLive Maskapai garuda indonesia min.
2      1:400 NG Model Garuda Indonesia BOEING 777-300...
3      KPK Mencecar Politikus Partai Demokrat Atte Su...
4      Apa Itu Pangkat Tituler? Ini 3 Pilot Garuda In...
                             ...                        
96     @pradinayashinta Selamat sore Kak, ada yang in...
97     Garuda Indonesia membuka kembali penerbangan i...
98     @priyayi_jogja @ilham_mochnoer @idextratime Ga...
99     @IntanAyuArimbi Kami nantikan penerbangan Anda...
100    Berkomitmen melayani penumpang pasca pandemi c...
Name: Tweet, Length: 101, dtype: object

In [18]:
from ahk.script.wordcld import WORDCLOUD

WRDCLOUD = WORDCLOUD(clean_text)

ModuleNotFoundError: No module named 'ahk'